In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import fbeta_score, make_scorer, accuracy_score,  confusion_matrix, cohen_kappa_score, precision_score, recall_score, classification_report, f1_score
from sklearn.preprocessing import OneHotEncoder,  StandardScaler
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor, AdaBoostClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

import optuna
import optuna.visualization as vis
import time

import scipy.stats as st

In [ ]:
heart_data = "../data/clean/resampled_data.csv"
heart_df = pd.read_csv(heart_data)
heart_df.head()

In [ ]:
heart_df.shape

In [ ]:
df_new = heart_df.drop(['PhysHlth', 'DiffWalk','Education'],  axis=1)
df_new

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=heart_df, x='HeartDiseaseorAttack')
plt.title(f'Count plot')
plt.xticks(rotation=45)  # Rotate x-axis labels if needed
plt.show()

In [ ]:
target = df_new['HeartDiseaseorAttack']
features = df_new.drop('HeartDiseaseorAttack', axis=1)

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

#Normalise all columns to be 0-1
normalizer = MinMaxScaler()
normalizer.fit(x_train)

x_train_norm = normalizer.transform(x_train)
x_test_norm = normalizer.transform(x_test)

x_train_norm = pd.DataFrame(x_train_norm, columns=x_train.columns, index=x_train.index )
x_test_norm = pd.DataFrame(x_test_norm, columns=x_test.columns, index=x_test.index)

#### Adaptive boosting

#### Using classifier

In [ ]:
ada_reg = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20),
                            n_estimators=100, algorithm='SAMME')
ada_reg.fit(x_train_norm, y_train)
pred = ada_reg.predict(x_test_norm)
print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"MSE, {mean_squared_error(pred, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(pred, y_test): .2f}")
print(f"R2 score, {ada_reg.score(x_test_norm, y_test): .2f}")

In [ ]:
def test_normalised_data(df_new):
    """
    Split the dataset into target and features, then divide it into train and test.
    Apply Min-Max Scaling to normalize the feature values to a range of 0 to 1 for both train and test.
    Iterates over a range of n_estimators (from 6 to 14) to train an AdaBoost classifier, decision tree base(max depth = 20)
    Uses the SAMME algorithm for boosting.
    For each iteration, calculates accuracy, recall, and Cohen's Kappa score.
    Stores the results into a dataframe.
    """
    
    target = df_new['HeartDiseaseorAttack']
    features = df_new.drop('HeartDiseaseorAttack', axis=1)

    x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)
    
    #Normalise all columns to be 0-1
    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    
    x_train_norm = normalizer.transform(x_train)
    x_test_norm = normalizer.transform(x_test)
    
    x_train_norm = pd.DataFrame(x_train_norm, columns=x_train.columns, index=x_train.index )
    x_test_norm = pd.DataFrame(x_test_norm, columns=x_test.columns, index=x_test.index)
    
    results = []

    
    # Iterate over a range of n_estimators for AdaBoost
    for i in range(2,25): 
        ada_boost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20),
            n_estimators=i,  # number of estimators (trees)
            algorithm='SAMME'
        )
        
        # Fit the AdaBoost model
        ada_boost.fit(x_train, y_train)
        # Predict the labels on the test set
        y_pred = ada_boost.predict(x_test)
        
        # Calculate Cohen's Kappa score
        kappa = cohen_kappa_score(y_test, y_pred)
    
        #Calculate the recall
        recall = recall_score(y_test, y_pred) * 100
        
        # Calculate accuracy manually using confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        accuracy = 100 * ((cm[0][0] + cm[1][1]) / (sum(cm[0]) + sum(cm[1])))

        print(f"AdaBoost Test MAE: {mean_absolute_error(y_test, y_pred): .4f}")
        print(f"AdaBoost Test MSE: {mean_squared_error(y_test, y_pred): .4f}")
        print(f"AdaBoost Test RMSE: {root_mean_squared_error(y_test, y_pred): .4f}")
        print(f"AdaBoost Test R2 score: {ada_boost.score(x_test_norm, y_test): .4f}")
        print()
        
        results.append({
            "k": i,
            "Accuracy": accuracy,
            "Recall": recall,
            "Kappa": kappa
        })
        results_df = pd.DataFrame(results)

    return results_df

In [ ]:
scores_df =test_normalised_data(df_new)
scores_df

#### Interpretation of the result

The MAE, MSE, and RMSE values suggest relatively small prediction errors, indicating that the model's predictions are quite close to the actual values.

#### General Performance:

- Accuracy improved with more neighbours, peaking at 74.70% in k=23 (estimators)
- Consistently improved, reaching a peak of 77.02% at k=20 making it critical for detecting true positives in heart attack risk prediction.
- Achieved its highest value of 0.4941 at k=23, indicating better agreement between predictions and actual values.

#### Best Configuration:

- 23 estimators provided the best results:
- Accuracy: 74.70%
- Recall: 76.94%
- Cohen’s Kappa: 0.4941

This configuration achieved the highest recall and Cohen's Kappa, balancing predictive accuracy and medical relevance.

Recall improvement is notable, reaching a peak of 77.02% at k=20, making it critical for detecting true positives in heart attack risk prediction.

In [ ]:
import plotly.express as px
# Melt the DataFrame for easier plotting with Plotly
df_melted = scores_df.melt(id_vars="k", value_vars=["Accuracy", "Recall"], 
                    var_name="Metric", value_name="Score")

# Create the line plot with Plotly Express
fig = px.line(df_melted, x="k", y="Score", color="Metric", markers=True,
              title="Model Performance Metrics vs. k (Number of Estimators)",
              labels={"k": "Number of Estimators (k)", "Score": "Metric Score"})

# Customize the layout for better presentation
fig.update_layout(
    title_font_size=20,
    xaxis_title="Number of Estimators (k)",
    yaxis_title="Metric Score in %",
    template="plotly_white",
    legend_title="Metrics",
    xaxis=dict(tickmode="linear"),
)

# Show the plot
fig.show()

#### Logistic regression

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(x_train_norm, y_train)

In [ ]:
log_reg.score(x_test_norm, y_test)

In [ ]:
pred = log_reg.predict(x_test_norm)
print(classification_report(y_pred = pred, y_true = y_test))

#### Grid search

In [ ]:
import time
import scipy.stats as st

# First we need to setup a dictionary with all the values that we want to try for each hyprerparameter

parameter_grid = {"max_depth": [10, 50],
                  "min_samples_split": [4, 16],
                  "max_leaf_nodes": [250, 100],
                  "max_features": ["sqrt", "log2"]} # In example we're going to test 2 * 2 * 2 * 2 = 16 combinations of hyperparameters

# We create an instance or our machine learning model
dt = DecisionTreeClassifier(random_state=123)

# We need to set this two variables to be able to compute a confidence interval
confidence_level = 0.95
folds = 10

# Now we need to create an intance of the GridSearchCV class
gs = GridSearchCV(dt, param_grid=parameter_grid, cv=folds, verbose=10) # Here the "cv" allows you to define the number of folds to use.

start_time = time.time()
gs.fit(x_train_norm, y_train)
end_time = time.time()

print("\n")
print(f"Time taken to find the best combination of hyperparameters among the given ones: {end_time - start_time: .4f} seconds")
print("\n")


print(f"The best combination of hyperparameters has been: {gs.best_params_}")
print(f"The R2 is: {gs.best_score_: .4f}")

results_gs_df = pd.DataFrame(gs.cv_results_).sort_values(by="mean_test_score", ascending=False)

#print(results_df.head())
gs_mean_score = results_gs_df.iloc[0,-3]
gs_sem = results_gs_df.iloc[0,-2] / np.sqrt(10)

gs_tc = st.t.ppf(1-((1-confidence_level)/2), df=folds-1)
gs_lower_bound = gs_mean_score - ( gs_tc * gs_sem )
gs_upper_bound = gs_mean_score + ( gs_tc * gs_sem )

print(f"The R2 confidence interval for the best combination of hyperparameters is: \
    ({gs_lower_bound: .4f}, {gs_mean_score: .4f}, {gs_upper_bound: .4f}) ")

#display(results_df)

# Let's store the best model
best_model = gs.best_estimator_

# Now is time evaluate the model in the test set
y_pred_test_df = best_model.predict(x_test_norm)
y_pred_test_df = best_model.predict(x_test_norm)

y_pred_test_df = best_model.predict(x_test_norm)

print("\n")
print(f"Test MAE: {mean_absolute_error(y_pred_test_df, y_test): .4f}")
print(f"Test MSE: {mean_squared_error(y_pred_test_df, y_test): .4f}")
print(f"Test RMSE: {root_mean_squared_error(y_pred_test_df, y_test): .4f}")
print(f"Test R2 score:  {best_model.score(x_test_norm, y_test): .4f}")
print("\n")



#### Bayesian search

In [ ]:
def objective(trial, confidence_level, folds):

    # First, we define the grid with values to consider when train several possible combinations.
    # Now we specify a range/list of values to try for each hyper-parameter, and we let optuna to decide which
    # combination to try.
    max_depth = trial.suggest_int("max_depth", 10, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 4, 16)
    max_leaf_nodes = trial.suggest_int("max_leaf_nodes", 250, 1000)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])

    dt = DecisionTreeClassifier(random_state=123,
                               max_depth=max_depth,
                               min_samples_split=min_samples_split,
                               max_leaf_nodes=max_leaf_nodes,
                               max_features=max_features)

    # Here the parameter "cv" specifies the number of folds K
    scores = cross_val_score(dt, x_train_norm, y_train, cv=folds) # The scores provided will be the R2 on each hold out fold
    mean_score = np.mean(scores)
    sem = np.std(scores, ddof=1) / np.sqrt(folds)

    tc = st.t.ppf(1-((1-confidence_level)/2), df=folds-1)
    lower_bound = mean_score - ( tc * sem )
    upper_bound = mean_score + ( tc * sem )

    # Here, we're storing confidence interval for each trial. It's not possible for the objective function to return
    # multiple values as Optuna uses the only returned value to find the best combination of hyperparameters.
    trial.set_user_attr("CV_score_summary", [round(lower_bound,4), round(np.mean(scores),4), round(upper_bound,4)])

    return np.mean(scores)

In [ ]:
confidence_level = 0.95
folds = 10

start_time = time.time()
study = optuna.create_study(direction="maximize") # We want to have the maximum values for the R2 scores
study.optimize(lambda trial: objective(trial, confidence_level, folds), n_trials=45)
end_time = time.time()

print("\n")
print(f"Time taken to find the best combination of hyperparameters among the given ones: {end_time - start_time: .4f} seconds")
print("\n")
print("The best combination of hyperparameters found was: ", study.best_params)
print(f"The best R2 found was: {study.best_value: .4f}")